# Crop planning
Farmer Jane owns __45 acres of land__.<br>
She is going to plant each with __wheat__ or __corn__.<br>
Each acre planted with wheat yields __\\$200__ profit; each with corn yields __\\$300__ profit.<br>
__The labor__ and __fertilizer__ used for each acre are given in the table below. <br>
__100 workers__ and __120 tons of fertilizer__ are available.

|&nbsp;| Wheat|Corn|
|:-------|-------:|-------:|
|Labor(workers)|3|2|
|Fertilizer(tons)|2|4|

How should Jane plant her crops to maximize profit?

Running the following code section will allow you to __manually__ choose the amount of corn and wheat that you will plant.<br>
Try to find the __maximum profit__.<br>
Then model and solve this problem using GAMS, check if the solution you found manually is optimal.

In [1]:
# Import the Python package for interactivity
import ipywidgets as widgets

# Load the GAMS extension
%load_ext gams_magic

def calc_total_acres(wheat_amount, corn_amount):
    print("{} + {} = {} Acres Planted.".format(wheat_amount, corn_amount, wheat_amount + corn_amount))
    if wheat_amount + corn_amount > 45:
        print("(Infeasible)")
    
def calc_total_workers(wheat_amount, corn_amount):
    print("3 * {} + 2 * {} = {} Worker used.".format(wheat_amount, corn_amount, (3 * wheat_amount) + (2 * corn_amount)))
    if (3 * wheat_amount) + (2 * corn_amount) > 100:
        print("(Infeasible)")
        
def calc_total_fertilizers(wheat_amount, corn_amount):
    print("2 * {} + 4 * {} = {} tons of Fertilizers used.".format(wheat_amount, corn_amount, (2 * wheat_amount) + (4 * corn_amount)))
    if (2 * wheat_amount) + (4 * corn_amount) > 120:
        print("(Infeasible)")
        
def calc_profit(wheat_amount, corn_amount):
    print("200 * {} + 300 * {} = $ {} profit.".format(wheat_amount, corn_amount, (200 * wheat_amount) + (300 * corn_amount)))

wheat_amount = widgets.IntSlider(value=0, min=0, max=45, description="Wheat(acres)")
corn_amount = widgets.IntSlider(value=0, min=0, max=45, description="Corn(acres)")

total_acres = widgets.interactive_output(calc_total_acres, {'wheat_amount': wheat_amount, 'corn_amount': corn_amount})
total_workers = widgets.interactive_output(calc_total_workers, {'wheat_amount': wheat_amount, 'corn_amount': corn_amount})
total_fertilizers = widgets.interactive_output(calc_total_fertilizers, {'wheat_amount': wheat_amount, 'corn_amount': corn_amount}) 
total_profit = widgets.interactive_output(calc_profit, {'wheat_amount': wheat_amount, 'corn_amount': corn_amount}) 


sliders = widgets.VBox([wheat_amount, corn_amount])
acres_and_workers_used =  widgets.VBox([total_acres, total_workers])
fertilizers_and_profit = widgets.VBox([total_fertilizers, total_profit])


widgets.VBox([widgets.HTML(value="<h3>Play with the sliders and see what profit you would get</h3>"), widgets.HBox([ sliders , acres_and_workers_used, fertilizers_and_profit])])

### Write a GAMS model and check your previous solution

For this script to work:<br>
- Make sure you named decision variable for acres of wheat to be planted as __WHEAT__<br>
- Make sure you named decision variable for acres of corn to be planted as __CORN__<br>
- Make sure you named decision variable for objective as __PI__<br>
- Or change pull statement below according to your naming convention<br>

After __%%gams__ you can type your GAMS model in the same cell.

In [3]:
%%gams
Set
    i 'inputs'  / labor, fertilizer /    
    j 'crops'  / WHEAT, CORN /;


Table t(i,j) 'production technology'
                        WHEAT    CORN
        labor           3        2
        fertilizer      2        4;


Parameter
    yields(j) 'yield of each acre planted with j'
        / WHEAT  200 
          CORN   300 /;

Variables
    WHEAT 'acres of wheat to be planted'
    CORN  'acres of corn to be planted'
    PI    'objective'
   
Positive Variable WHEAT, CORN;  

Equation
    obj 'calculate the profits',
    lands_eq 'planted lands must be less than lands owned by Jane',
    workers_eq 'workers must be less than the total',  
    fertilizer_eq 'fertilizer must be less than the total';

    lands_eq.. WHEAT + CORN =l= 45; 
    workers_eq.. WHEAT*t('labor','WHEAT') + CORN*t('labor','CORN') =l= 100; 
    fertilizer_eq.. WHEAT*t('fertilizer','WHEAT') + CORN*t('fertilizer','CORN') =l= 120; 
    obj.. WHEAT*yields('WHEAT') + CORN*yields('CORN') =e= PI;

model prob /all/;

Options LP  = Cplex;

Solve prob using LP maximizing PI;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),10000.0,4,3,LP,CPLEX,0


In [4]:
# Pull variable values from GAMS to Python
%gams_pull wheat corn pi
print("Acres of wheat computed by GAMS: " + str(wheat[0][0]))
print("Acres of corn computed by GAMS: " + str(corn[0][0]))
print("Maximum profit achieveable is: $" + str(pi[0][0]))

Acres of wheat computed by GAMS: 20.0
Acres of corn computed by GAMS: 20.0
Maximum profit achieveable is: $10000.0


### Solve the problem graphically and confirm that you obtain the same solution

In [1]:
# imports for plotting
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# This function can be provided to students
def abline(slope, intercept, label):
    """Plot a line from slope and intercept"""
    axes = plt.gca()
    x_vals = np.array(axes.get_xlim())
    y_vals = intercept + slope * x_vals
    plt.plot(x_vals, y_vals, label=label)

# This function can also be provided to students
def axbyc(a, b, c, label):
    abline(-a/b ,c/b, label)

# Students can be asked to fill xlim and ylim
fig, ax = plt.subplots()
plt.xlim((0, 60))   
plt.ylim((0, 50))

axbyc(3, 2, 100, "Workers")
axbyc(2, 4, 120, "Fertilizers")
axbyc(1, 1, 45, "Land")
axbyc(200, 300, 10000, "Objective")

# You can use ax.fill(x_coords, y_coords)
# to fill a polygon
ax.fill([0,20,33,0], [30,20,0,0], "g") 

plt.legend(loc='upper right')
plt.show()

UsageError: Line magic function `%gams_pull` not found.
